In [312]:
import numpy as np
import os
import time
import datetime as dt
import torch
import torch.nn as nn
import torch.optim as optim, lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms, datasets
from torch.autograd import Variable
import torchvision

In [257]:
# Send the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [298]:
# Define transforms to be applied to the image data
transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(100),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [299]:
# Load the dataset
train_dataset = datasets.ImageFolder("D:/tay/Data/Data_crop/Data_split/classes_image/image_train/", transform=transform)

# Define the data loader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define the label names
label_names = train_dataset.classes

In [313]:
# Load the dataset
val_dataset = datasets.ImageFolder("D:/tay/Data/Data_crop/Data_split/classes_image/image_test/", transform=transform)

# Define the data loader
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

# Define the label names
label_names = val_dataset.classes

In [80]:
print(label_names)

['hand_A', 'hand_A2', 'hand_B', 'hand_C', 'hand_D', 'hand_D2', 'hand_E', 'hand_G', 'hand_H', 'hand_I', 'hand_K', 'hand_L', 'hand_M', 'hand_N', 'hand_O', 'hand_O3', 'hand_P', 'hand_Q', 'hand_R', 'hand_S', 'hand_T', 'hand_U', 'hand_V', 'hand_X', 'hand_Y']


In [327]:
def train_model(dataloaders, model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    use_gpu = torch.cuda.is_available()
    best_model_wts = model.state_dict()
    best_acc = 0.0
    dataset_sizes = {'train': len(train_loader.dataset), 
                     'valid': len(val_loader.dataset)}

    for epoch in range(num_epochs):
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                optimizer.zero_grad()

                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                train_epoch_loss = running_loss / dataset_sizes[phase]
                train_epoch_acc = running_corrects / dataset_sizes[phase]
            else:
                valid_epoch_loss = running_loss / dataset_sizes[phase]
                valid_epoch_acc = running_corrects / dataset_sizes[phase]
                
            if phase == 'valid' and valid_epoch_acc > best_acc:
                best_acc = valid_epoch_acc
                best_model_wts = model.state_dict()

        print('Epoch [{}/{}] train loss: {:.4f} acc: {:.4f} ' 
              'valid loss: {:.4f} acc: {:.4f}'.format(
                epoch, num_epochs - 1,
                train_epoch_loss, train_epoch_acc, 
                valid_epoch_loss, valid_epoch_acc))
            
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [328]:
resnet = models.resnet101(pretrained=True)
# freeze all model parameters
for param in resnet.parameters():
    param.requires_grad = False
use_gpu = torch.cuda.is_available()
# new final layer with 25 classes
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, 25)
if use_gpu:
    resnet = resnet.cuda()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet.fc.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

dloaders = {'train':train_loader, 'valid':val_loader}

In [ ]:
start_time = time.time()
model = train_model(dloaders, resnet, criterion, optimizer, exp_lr_scheduler, 2)
print('Training time: {:10f} minutes'.format((time.time()-start_time)/60))

Epoch [0/1] train loss: 0.0789 acc: 0.6224 valid loss: 0.1330 acc: 0.4642
